# Contexte :

On va réaliser un mini bot analyst. Il sera chargé d'analyser les données d'une entreprise et de l'aider dans la prise de décision

# packages

In [ ]:
!pip install langchain
!pip install langchain_google_genai
! pip install langchain_chroma
!pip install langchain_community
!pip install -qU langchain-text-splitters

# Variables d'environnement

In [2]:
import os

import getpass

os.environ["GOOGLE_API_KEY"]= getpass.getpass()

··········


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.1)

# Chargement du csv

In [4]:
csv_path="/content/drive/MyDrive/Colab Notebooks/tips.csv"

In [5]:
#Ici, on va charger le fichier csv : il va contenir des infos spécifiques sur l'entreprise

from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path=csv_path)

data = loader.load()

In [6]:
len(data)

244

In [7]:
data[0]

Document(page_content='total_bill: 16.99\ntip: 1.01\nsex: Female\nsmoker: No\nday: Sun\ntime: Dinner\nsize: 2', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/tips.csv', 'row': 0})

In [8]:
data[105]

Document(page_content='total_bill: 15.36\ntip: 1.64\nsex: Male\nsmoker: Yes\nday: Sat\ntime: Dinner\nsize: 2', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/tips.csv', 'row': 105})

# Découpage des données

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=300,
   length_function=len,
)

In [11]:
data_split=splitter.split_documents(data)

In [12]:
len(data)

244

In [13]:
data[200]

Document(page_content='total_bill: 18.71\ntip: 4.0\nsex: Male\nsmoker: Yes\nday: Thur\ntime: Lunch\nsize: 3', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/tips.csv', 'row': 200})

# Embedding

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [15]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", temperature=0.1)

In [16]:
from langchain_chroma import Chroma

In [17]:
data_store=Chroma.from_documents(documents=data_split,embedding=embeddings)

# Questionnnement de la bdv

In [18]:
retriever = data_store.as_retriever(search_type="similarity", search_kwargs={"k": 20})



In [19]:
from langchain.prompts import PromptTemplate

In [20]:
#Je crée ici un template à utiliser pour questionner le modèle.

template=""""Tu es un bot  expert en statistiques, en analyse de données et en décisions probabilistes. Tu es chargé d'aider les entreprises dans
l'analyse de leurs données et dans la prise de décision.

Ton nom est Babel, le bot data analyst qui aide les entreprises dans leur prise de décisions stratégiques.

Comporte toi comme un data analyst.



Tu recevras un fichier contenant des informations sur l'entreprise. ça peut être des infos sur les résultats de la compagne marketing,
le chiffre d'affaires, les informations sur les ventes, les dépenses au sein de l'entreprise.


Utilise tes connaissances en statistiques, analyse de données, probabilités, prisse de décision, recherche opérationnelle,optimisation,
marketing, comptabilité et finances en tant que gemini pro pour faire des analyses statistiques et fournir des réponses adaptées aux préoccupations.



-Tu feras le nettoyage du dataset;

-Tu analyseras le dataset en termes de taille , identifier les types de données pour voir avec quelle type de données(numérique ou catégorielle), tu travailles;

-Tu feras le traitement des variables manquantes;

-Tu feras les analyses univariées et bivariées pour analyser les distributions des variables,explorer les relations entre ces dernières.

- Après celà tu pourras répondre à la préoccupation de l'entreprise en usant de tes connaissances en data storytelling, pour communiquer de façon claire, détaillée et précise.



Voici la préoccupation : {input}

  context: {context}
  input: {input}
  answer:


"""


prompt=PromptTemplate.from_template(template)

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain

combine_docs_chain = create_stuff_documents_chain(llm, prompt)

In [22]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [23]:
response=retrieval_chain.invoke({"input":"Qui es tu ?"})


In [24]:
print(response['answer'])

**Nettoyage du jeu de données**

Le jeu de données fourni ne nécessite aucun nettoyage.

**Analyse du jeu de données**

**Taille et types de données**

Le jeu de données contient 20 observations et 7 variables :

* **total_bill** : Montant total de la facture (numérique)
* **tip** : Montant du pourboire (numérique)
* **sex** : Sexe du client (catégorielle)
* **smoker** : Si le client est fumeur ou non (catégorielle)
* **day** : Jour de la semaine (catégorielle)
* **time** : Moment de la journée (catégorielle)
* **size** : Nombre de personnes dans le groupe (numérique)

**Traitement des variables manquantes**

Aucune variable manquante n'est présente dans le jeu de données.

**Analyses univariées**

**total_bill**

* Moyenne : 25,68
* Écart-type : 10,46
* Minimum : 9,68
* Maximum : 48,17

**tip**

* Moyenne : 4,17
* Écart-type : 1,68
* Minimum : 1,32
* Maximum : 6,00

**sex**

* Homme : 16 observations
* Femme : 4 observations

**smoker**

* Non : 17 observations
* Oui : 3 observations
